In [ ]:
#### Lane Detection Algorithm

In [14]:
import numpy as np
import cv2

In [ ]:
#Question 1
image = cv2.imread(""/Deep-Learning/Module 1 - Steps to Deep Learning/Extra Class - ComputerVision Using OpenCV/Lane Detection/exercise/dataset/lane_single_image.jpg"")
grayscale = # ************** your code here *******************
print("total intensities: ", sum(sum(grayscale)))

total intensities:  162333


In [16]:
# Question 2
kernel_size = 5
blur = # ************** your code here *******************
print("total intensities: ", sum(sum(blur)))

total intensities:  159945


In [17]:
# Question 3
low_t = 50
high_t = 150
edges = # ************** your code here *******************
print("total intensities: ", sum(sum(edges)))

total intensities:  318588


In [18]:
# Question 4
def limit_research_space(image):
	mask = np.zeros_like(image)
	ignore_mask_color = 255
	# creating a polygon to focus only on the road in the picture
	# we have created this polygon in accordance to how the camera was placed
	rows, cols = image.shape[:2]
	bottom_left = [cols * 0.1, rows * 0.95]
	top_left	 = [cols * 0.4, rows * 0.6]
	bottom_right = [cols * 0.9, rows * 0.95]
	top_right = [cols * 0.6, rows * 0.6]
	vertices = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)
	# filling the polygon with white color and generating the final mask
	cv2.fillPoly(mask, vertices, ignore_mask_color)
	# performing Bitwise AND on the input image and mask to get only the edges on the road
	masked_image = cv2.bitwise_and(image, mask)
	return masked_image
region = limit_research_space(edges)
print("total intensities: ", sum(sum(region)))

total intensities:  104652


In [19]:
# Question 5: Use hough transform to detect left and right lane
def hough_transform(image):
	"""
	Determine and cut the region of interest in the input image.
	Parameter:
		image: grayscale image which should be an output from the edge detector
	"""
	# Distance resolution of the accumulator in pixels.
	rho = 1
	# Angle resolution of the accumulator in radians.
	theta = np.pi/180
	# Only lines that are greater than threshold will be returned.
	threshold = 20
	# Line segments shorter than that are rejected.
	minLineLength = 20
	# Maximum allowed gap between points on the same line to link them
	maxLineGap = 500
	# function returns an array containing dimensions of straight lines
	# appearing in the input image
	return cv2.HoughLinesP(image, rho = rho, theta = theta, threshold = threshold,
						minLineLength = minLineLength, maxLineGap = maxLineGap)

hough = hough_transform(region)
print("Dimensions of straight lines", hough.shape)

Dimensions of straight lines (17, 1, 4)


In [20]:
# Question 6: Draw the left and right lane to the screen

def average_slope_intercept(lines):
	"""
	Find the slope and intercept of the left and right lanes of each image.
	Parameters:
		lines: output from Hough Transform
	"""
	left_lines = [] #(slope, intercept)
	left_weights = [] #(length,)
	right_lines = [] #(slope, intercept)
	right_weights = [] #(length,)

	for line in lines:
		for x1, y1, x2, y2 in line:
			if x1 == x2:
				continue
			# calculating slope of a line
			slope = (y2 - y1) / (x2 - x1)
			# calculating intercept of a line
			intercept = y1 - (slope * x1)
			# calculating length of a line
			length = np.sqrt(((y2 - y1) ** 2) + ((x2 - x1) ** 2))
			# slope of left lane is negative and for right lane slope is positive
			if slope < 0:
				left_lines.append((slope, intercept))
				left_weights.append((length))
			else:
				right_lines.append((slope, intercept))
				right_weights.append((length))
	#
	left_lane = np.dot(left_weights, left_lines) / np.sum(left_weights) if len(left_weights) > 0 else None
	right_lane = np.dot(right_weights, right_lines) / np.sum(right_weights) if len(right_weights) > 0 else None
	return left_lane, right_lane


def pixel_points(y1, y2, line):
	"""
	Converts the slope and intercept of each line into pixel points.
		Parameters:
			y1: y-value of the line's starting point.
			y2: y-value of the line's end point.
			line: The slope and intercept of the line.
	"""
	if line is None:
		return None
	slope, intercept = line
	x1 = int((y1 - intercept)/slope)
	x2 = int((y2 - intercept)/slope)
	y1 = int(y1)
	y2 = int(y2)
	return ((x1, y1), (x2, y2))

def lane_lines(image, lines):
	"""
	Create full lenght lines from pixel points.
		Parameters:
			image: The input test image.
			lines: The output lines from Hough Transform.
	"""
	left_lane, right_lane = average_slope_intercept(lines)
	y1 = image.shape[0]
	y2 = y1 * 0.6
	left_line = pixel_points(y1, y2, left_lane)
	right_line = pixel_points(y1, y2, right_lane)
	return left_line, right_line


def draw_lane_lines(image, lines, color=[0, 255, 0], thickness=12):
	"""
	Draw lines onto the input image.
		Parameters:
			image: The input test image (video frame in our case).
			lines: The output lines from Hough Transform.
			color (Default = red): Line color.
			thickness (Default = 12): Line thickness.
	"""
	line_image = np.zeros_like(image)
	for line in lines:
		if line is not None:
			cv2.line(line_image, *line, color, thickness)
	return cv2.addWeighted(image, 1.0, line_image, 1.0, 0.0)

result = draw_lane_lines(image, lane_lines(image, hough))
cv2.imshow("result",result)
cv2.waitKey(0)
close_window_os()
print("Image result's resolution", result.shape)

Image result's resolution (704, 1279, 3)


In [ ]:
#Question 7
#Lane detection for video image
import cv2
cap = cv2.VideoCapture("/Users/nguyendinhvinh2004gmail.com/Desktop/New_Data/De Cuong mon học/Machine Learning Course - VinhND/Week 4/QSTC/Computer Vision Foundation/Computer Vision Foundation - Session 2 - LD/dataset/test2.mp4")

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    # if frame is read correctly ret is True
    
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
        
    # ************** your code here *******************
    
    if cv2.waitKey(1) == ord('q'):
        break
    
close_window_os()

In [4]:
def close_window_os():
    cv2.destroyAllWindows()
    for i in range (1,5): #trick for closing the display window on macos
        cv2.waitKey(1)